In [1]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

In [2]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}

In [4]:
DATE_STOP = '2021-12-21'

In [5]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [67]:
def naver_crawl(ACODE):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    pages = [1]
    title_result=[]
   
 
    global DATE_STOP
    #seen_date = db.naver_title_crawl_1114.find({"code":acode}).sort('date', -1).limit(1)
    
    #sort 전 어떤 변수에 저장해서, 
    #코드 기준으로 가져와서 데이트 정렬
    #seen_date = list(seen_date)
    #if not seen_date:
        
    seen_date = "2021.12.21 00:00"
    TF = False

    for page in pages:
            #title_result=[]
        pages.append(page + 1)
        url = 'https://finance.naver.com/item/board.nhn?code={}&page={}'.format(ACODE,page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        table = dom.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')


        for k in tt[2:]:
            if len(k.select('td > span')) > 0:
                date = k.select('td > span')[0].text
                if str(date)<=str(seen_date) or date[:10]==DATE_STOP:
                    TF = True
                    break

                title = k.select('td.title > a')[0]['title']
                views = k.select('td > span')[1].text
                pos = k.select('td > strong')[0].text
                neg = k.select('td > strong')[1].text


                put_data = {
                                     'code':ACODE,
                                     'date':date,
                                      'title':title,
                                      'views':views,
                                      'pos':pos,
                                       'neg':neg
                                                 }

                title_result.append(put_data)
   

        if TF:
            break
            return title_result
        else:
            
            print(title_result)


In [ ]:
for v in ACODE.keys():
    print(naver_crawl(v))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
